# Crear Target

## Ejercicio 1:
Cree el archivo `competencia_01.csv`, usando el `competencia_01_crudo.csv` con una variable adicional llamada `clase_ternaria`, que contenga las categorías **CONTINUA**, **BAJA+1** y **BAJA+2** 

### Ayudita 

Para practicar el muy útil y necesario lenguaje **SQL**, vamos a utilizar una base de datos **OLAP** llamada **DuckDB**. 

La documentación la puede encontrar [aquí](https://duckdb.org/docs/archive/0.8.1/sql/introduction)
Procedemos a instalarla, esto se debe ejecutar una sola vez

In [2]:
# %%bash 
# pip install duckdb
# pip install jupysql
# pip install duckdb-engine

Configuramos el entorno de ejecución. Si ya tiene todo instalado, solo necesita ejecutar esta celda para empezar a usar **duckdb** 

In [3]:
# %%bash 
# pip install pandas

In [4]:
import duckdb
import pandas as pd

%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

%sql duckdb:///:default:

Y ya podemos usar **SQL** dentro de una notebook!

In [5]:
%%sql
select 'hola mundo' 

,'hola mundo'
0,hola mundo


Para cargar el archivo `.csv` a una tabla:

In [6]:
%%sql
create or replace table competencia_01 as 
select *
  from read_csv_auto('/Users/mkiszkurno/Documents/dmeyf/datasets/competencia_01_crudo.csv')
  
  

,Success


Hagamos unas queries básicas para comprobar que todo esta funcionando bien.

In [7]:
%sql select * from competencia_01 limit 5

,numero_de_cliente,foto_mes,active_quarter,cliente_vip,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,...,Visa_madelantodolares,Visa_fultimo_cierre,Visa_mpagado,Visa_mpagospesos,Visa_mpagosdolares,Visa_fechaalta,Visa_mconsumototal,Visa_cconsumos,Visa_cadelantosefectivo,Visa_mpagominimo
0,29183981,202103,1,0,0,50,197,14468.81,125765.29,2389.82,...,0.0,7,0.00,-114954.00,0.00,5938,101050.66,68,0,71811.06
1,29184630,202103,1,0,0,59,322,11901.57,74158.93,18750.68,...,0.0,1,0.00,-40330.15,17.59,4089,26834.09,7,0,3894.36
2,29185433,202103,1,0,0,68,268,847.15,21672.47,481.62,...,0.0,21,4692.00,-1173.00,0.00,7829,1651.36,3,0,1560.09
3,29185587,202103,1,0,0,79,322,4976.94,47735.98,1839.31,...,0.0,1,0.00,-15988.67,0.00,7580,30025.29,11,0,1700.85
4,29185646,202103,1,0,0,60,257,2860.45,37800.71,4035.40,...,0.0,21,380616.14,-97383.25,0.00,7827,359610.70,31,0,15600.90


In [8]:
%%sql
select foto_mes, 
       count(*) as cantidad -- cuenta cuantos casos hay en cada foto_mes 
                           -- y lo guarda en un campo llamado cantidad
  from competencia_01
 group by foto_mes

,foto_mes,cantidad
0,202103,163324
1,202104,163637
2,202105,164102


Perfecto, ahora cree una nueva tabla con la variable adicional que se le pide. Algunas funciones que le pueden ser útiles:  [where](https://duckdb.org/docs/sql/query_syntax/where), [left join](https://duckdb.org/docs/sql/query_syntax/from), [case statement](https://duckdb.org/docs/sql/expressions/case)



In [9]:
%%sql

SELECT numero_de_cliente, foto_mes, 'CONTINUA' as clase_ternaria
  FROM competencia_01 C
 WHERE C.foto_mes = 202103 
   and EXISTS (SELECT numero_de_cliente FROM competencia_01 C2 WHERE C.numero_de_cliente = C2.numero_de_cliente AND C2.foto_mes = C.foto_mes + 1)
   and EXISTS (SELECT numero_de_cliente FROM competencia_01 C3 WHERE C.numero_de_cliente = C3.numero_de_cliente AND C3.foto_mes = C.foto_mes + 2)






,numero_de_cliente,foto_mes,clase_ternaria
0,117250370,202103,CONTINUA
1,117250712,202103,CONTINUA
2,117251164,202103,CONTINUA
3,117251736,202103,CONTINUA
4,117253274,202103,CONTINUA
...,...,...,...
161306,117246006,202103,CONTINUA
161307,117246493,202103,CONTINUA
161308,117246911,202103,CONTINUA
161309,117247748,202103,CONTINUA


In [10]:
%%sql

SELECT numero_de_cliente, foto_mes, 'BAJA+1' as clase_ternaria
  FROM competencia_01 C
 WHERE C.foto_mes = 202103 
   and NOT EXISTS (SELECT numero_de_cliente FROM competencia_01 C2 WHERE C.numero_de_cliente = C2.numero_de_cliente AND C2.foto_mes = C.foto_mes + 1)
   --and NOT EXISTS (SELECT numero_de_cliente FROM competencia_01 C3 WHERE C.numero_de_cliente = C3.numero_de_cliente AND C3.foto_mes = C.foto_mes + 2)






,numero_de_cliente,foto_mes,clase_ternaria
0,117307441,202103,BAJA+1
1,117361993,202103,BAJA+1
2,117406374,202103,BAJA+1
3,117594513,202103,BAJA+1
4,117994309,202103,BAJA+1
...,...,...,...
1045,116788661,202103,BAJA+1
1046,116793272,202103,BAJA+1
1047,116802862,202103,BAJA+1
1048,117098346,202103,BAJA+1


In [11]:
%%sql

SELECT numero_de_cliente, foto_mes, 'BAJA+2' as clase_ternaria
  FROM competencia_01 C
 WHERE C.foto_mes = 202103 
   and EXISTS (SELECT numero_de_cliente FROM competencia_01 C2 WHERE C.numero_de_cliente = C2.numero_de_cliente AND C2.foto_mes = C.foto_mes + 1)
   and NOT EXISTS (SELECT numero_de_cliente FROM competencia_01 C3 WHERE C.numero_de_cliente = C3.numero_de_cliente AND C3.foto_mes = C.foto_mes + 2)




,numero_de_cliente,foto_mes,clase_ternaria
0,117291062,202103,BAJA+2
1,117349892,202103,BAJA+2
2,117475426,202103,BAJA+2
3,117561934,202103,BAJA+2
4,117606477,202103,BAJA+2
...,...,...,...
958,116731402,202103,BAJA+2
959,116942530,202103,BAJA+2
960,116964084,202103,BAJA+2
961,117009088,202103,BAJA+2


In [12]:
%%sql
SELECT numero_de_cliente, foto_mes, 'CONTINUA' as clase_ternaria
  FROM competencia_01 C
 WHERE C.numero_de_cliente = 122484758 


,numero_de_cliente,foto_mes,clase_ternaria
0,122484758,202103,CONTINUA
1,122484758,202104,CONTINUA


In [13]:
%%sql
SELECT numero_de_cliente, foto_mes, 'BAJA+1' as clase_ternaria
  FROM competencia_01 C
 WHERE C.foto_mes = 202103 
   and NOT EXISTS (SELECT numero_de_cliente FROM competencia_01 C2 WHERE C.numero_de_cliente = C2.numero_de_cliente AND C2.foto_mes = C.foto_mes + 1)


,numero_de_cliente,foto_mes,clase_ternaria
0,117307441,202103,BAJA+1
1,117361993,202103,BAJA+1
2,117406374,202103,BAJA+1
3,117594513,202103,BAJA+1
4,117994309,202103,BAJA+1
...,...,...,...
1045,116788661,202103,BAJA+1
1046,116793272,202103,BAJA+1
1047,116802862,202103,BAJA+1
1048,117098346,202103,BAJA+1


In [14]:
%%sql

CREATE OR REPLACE TABLE COMPETENCIA AS

SELECT C.*, 'CONTINUA' as clase_ternaria 
  FROM competencia_01 C
 WHERE C.foto_mes = 202103 
   and EXISTS (SELECT numero_de_cliente FROM competencia_01 C2 WHERE C.numero_de_cliente = C2.numero_de_cliente AND C2.foto_mes = C.foto_mes + 1)
   and EXISTS (SELECT numero_de_cliente FROM competencia_01 C3 WHERE C.numero_de_cliente = C3.numero_de_cliente AND C3.foto_mes = C.foto_mes + 2)

UNION

SELECT C.*, 'BAJA+1' as clase_ternaria
  FROM competencia_01 C
 WHERE C.foto_mes = 202103 
   and NOT EXISTS (SELECT numero_de_cliente FROM competencia_01 C2 WHERE C.numero_de_cliente = C2.numero_de_cliente AND C2.foto_mes = C.foto_mes + 1)

UNION

SELECT C.*, 'BAJA+2' as clase_ternaria
  FROM competencia_01 C
 WHERE C.foto_mes = 202103 
   and EXISTS (SELECT numero_de_cliente FROM competencia_01 C2 WHERE C.numero_de_cliente = C2.numero_de_cliente AND C2.foto_mes = C.foto_mes + 1)
   and NOT EXISTS (SELECT numero_de_cliente FROM competencia_01 C3 WHERE C.numero_de_cliente = C3.numero_de_cliente AND C3.foto_mes = C.foto_mes + 2)

UNION

SELECT C.*, null as clase_ternaria
  FROM competencia_01 C
 WHERE C.foto_mes = 202104 

UNION

SELECT C.*, null as clase_ternaria
  FROM competencia_01 C
 WHERE C.foto_mes = 202105 


,Success


In [71]:
%%sql
select foto_mes, clase_ternaria, count(*) from competencia2
where foto_mes = '202103'
group by foto_mes, clase_ternaria

,foto_mes,clase_ternaria,count_star()
0,202103,CONTINUA,161311
1,202103,BAJA+1,1049
2,202103,BAJA+2,964


In [16]:
%%sql
select foto_mes, 
       count(*) as cantidad -- cuenta cuantos casos hay en cada foto_mes 
                           -- y lo guarda en un campo llamado cantidad
  from competencia
 group by foto_mes

,foto_mes,cantidad
0,202103,163324
1,202104,163637
2,202105,164102


## Ejercicio 1.1

* ¿Cuál es la nominalidad de cada clase?
* ¿Cuál es la proporción del target?

In [17]:
%%sql
-- introduzca sus queries

,Success


Para guardar a un **.csv** simplemente debe ejecutar la siguiente sentencia

In [20]:
%%sql
--COPY competencia_01 TO '/home/aleb/dmeyf23/datasets/competencia_01.csv' (FORMAT CSV, HEADER)
COPY COMPETENCIA TO '/Users/mkiszkurno/Documents/dmeyf/datasets/competencia_01.csv' WITH (HEADER 1, DELIMITER ',');


,Success


## Ejercicio 2 - Avanzado
Use ahora el archivo `ejercicio_target.csv` y calcule para todos los clientes en todos los periodos su **clase_ternaria** al mismo tiempo

In [39]:
%%sql
select numero_de_cliente, foto_mes, row_number() over (partition by numero_de_cliente order by foto_mes) as clase_ternaria
from competencia_01
order by 1,2

,numero_de_cliente,foto_mes,clase_ternaria
0,29183981,202103,1
1,29183981,202104,2
2,29183981,202105,3
3,29184630,202103,1
4,29184630,202104,2
...,...,...,...
491058,186790659,202105,1
491059,186792315,202105,1
491060,186795526,202105,1
491061,186798379,202105,1


In [62]:
%%sql
select 
numero_de_cliente,
foto_mes,
row_number() over (partition by numero_de_cliente order by foto_mes),
lead(numero_de_cliente, 1) over (partition by numero_de_cliente order by foto_mes) as mes_1, 
lag(numero_de_cliente, 1) over (partition by numero_de_cliente order by foto_mes) as otro,
lag(numero_de_cliente, 2) over (partition by numero_de_cliente order by foto_mes) as otro2,
lag(foto_mes, 3) over (partition by numero_de_cliente order by foto_mes) as otro3,
nth_value(numero_de_cliente, 1) over (partition by numero_de_cliente order by foto_mes),
nth_value(numero_de_cliente, 2) over (partition by numero_de_cliente order by foto_mes)

from competencia_01
order by 1,2


,numero_de_cliente,foto_mes,row_number() OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes),mes_1,otro,otro2,otro3,"nth_value(numero_de_cliente, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes)","nth_value(numero_de_cliente, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes)"
0,29183981,202103,1,29183981.0,NaN,NaN,NaN,29183981,NaN
1,29183981,202104,2,29183981.0,29183981.0,NaN,NaN,29183981,29183981.0
2,29183981,202105,3,NaN,29183981.0,29183981.0,NaN,29183981,29183981.0
3,29184630,202103,1,29184630.0,NaN,NaN,NaN,29184630,NaN
4,29184630,202104,2,29184630.0,29184630.0,NaN,NaN,29184630,29184630.0
...,...,...,...,...,...,...,...,...,...
491058,186790659,202105,1,NaN,NaN,NaN,NaN,186790659,NaN
491059,186792315,202105,1,NaN,NaN,NaN,NaN,186792315,NaN
491060,186795526,202105,1,NaN,NaN,NaN,NaN,186795526,NaN
491061,186798379,202105,1,NaN,NaN,NaN,NaN,186798379,NaN


In [67]:
   
%%sql
CREATE OR REPLACE TABLE COMPETENCIA2 AS

SELECT
        numero_de_cliente,
        foto_mes,
        LEAD(foto_mes, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS mes2,
        LEAD(foto_mes, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS mes3,
        CASE
            WHEN LEAD(foto_mes, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) IS NOT NULL
                 AND LEAD(foto_mes, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) IS NOT NULL
            THEN 'CONTINUA'

            WHEN LEAD(foto_mes, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) IS NULL
            THEN 'BAJA+1'

            WHEN LEAD(foto_mes, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) IS NOT NULL
                 AND LEAD(foto_mes, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) IS NULL
            THEN 'BAJA+2'

        END AS clase_ternaria
 from competencia_01
order by 1,2


,Success


In [70]:
%%sql
select foto_mes, clase_ternaria, count(*) from competencia2
where foto_mes = '202103'
group by foto_mes, clase_ternaria

,foto_mes,clase_ternaria,count_star()
0,202103,CONTINUA,161311
1,202103,BAJA+1,1049
2,202103,BAJA+2,964
